# How to breakdown a complex ML project into manageable steps

## STEP 1: Device Check: CPU or GPU
This is important to ensure that our code runs on the appropriate hardware, especially if we're using a GPU for acceleration.

In [1]:
# Install libraries if not already installed
! pip install torch

# Clear output here to avoid clutter
from IPython.display import clear_output
clear_output()

In [3]:
# STEP 1: Device Check: CPU or GPU
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## STEP 2: 
- Now we know about our device, let's focus on the algorithm that we want to implement.
- So, let's say we want to implement a simple GANs (Generative Adversarial Networks) using PyTorch.
- What I know about GANs: They are consist of two main components: a Generator and a Discriminator.

![gan image](https://i.imgur.com/Ed5ZMfR.png)

<p align = "center">
Fig.1 - GAN Architecture  
(<a href="https://github.com/JamesAllingham/LaTeX-TikZ-Diagrams">
source
</a>)
</p>

## STEP 3:
- Okay, I know the overall architecture of GANs now, but I need to understand how to implement it in PyTorch.
- First let's find the dataset that we want to use.
- I know Kaggle is a great place to find datasets, so let's search for "GAN datasets" on Kaggle.
- I found this [link](https://www.kaggle.com/datasets/splcher/animefacedataset) which has a collection of Anime images that we can use to train our GAN.
- Let's download the dataset and extract it to a folder named `data`.
- But I might need to preprocess the data before using it.

In [4]:
# So we need a custom dataset class for anime images.
# But I would need some libraries for that.
! pip install pillow
clear_output()


In [23]:
# Now let's import the necessary libraries
import os
from torch.utils.data import Dataset
from PIL import Image
from glob import glob

In [ ]:
# Lets build out custom dataset class for anime images
class AnimeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """Custom dataset for loading anime images from a directory.
        Args:
            root_dir (str): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on an image.
        """
        self.root_dir = root_dir
        self.transform = transform
        # Get all image files
        self.image_files = []
        for ext in ['*.jpg', '*.jpeg', '*.png']:
            self.image_files.extend(glob(os.path.join(root_dir, '**', ext), recursive=True))
        print(f"Found {len(self.image_files)} images")
    

    def __len__(self):
        """Return the total number of images."""
        return len(self.image_files)

    def __getitem__(self, idx):
        """Load and return an image at the given index.
        Args:
            idx (int): Index of the image to retrieve.
        Returns:
            image (PIL Image or Tensor): Loaded image after applying transforms.
        """
        img_name = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

In [7]:
# Since I'm using Kaggle, I need to install `kagglehub` to download datasets
! pip install kagglehub
clear_output()

In [8]:
# Now lets define the dataset path and download the dataset
dataset_path = "./data"
if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)


In [14]:
# Lets import kagglehub and download the dataset
import kagglehub

# In order to use kagglehub, we need to set up our Kaggle API credentials.
# Follow these steps: https://www.kaggle.com/general/74235
# or this instruction: https://github.com/Kaggle/kagglehub
# Make sure to upload your `kaggle.json` file to `~/.kaggle/kaggle.json`.


path = kagglehub.dataset_download("splcher/animefacedataset")

100%|██████████| 395M/395M [00:14<00:00, 29.3MB/s] 

Extracting files...


In [15]:
print(f"Dataset downloaded to: {path}")

Dataset downloaded to: /home/prashant/.cache/kagglehub/datasets/splcher/animefacedataset/versions/3


In [18]:
# Let's check the size of a sample image from the dataset
sample_image_path = os.path.join(path, "images/10000_2004.jpg")
with Image.open(sample_image_path) as img:
    print(f"Sample image size: {img.size}")  # Output the size of the image

Sample image size: (62, 62)


In [19]:
# Lets resize it to 64x64 for our GAN model
IMG_SIZE = 64

# But also at the same time, we need to convert it to a tensor
# And also normalize it to the range [-1, 1]

# For this we want to use `transforms` from `torchvision`

In [20]:
# Lets install torchvision if not already installed
! pip install torchvision
clear_output()

In [21]:
# Now we are ready to import transforms and then define our transformations for the dataset in sequence
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),  # Resize to 64x64
    transforms.ToTensor(),                     # Convert to Tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

In [42]:
# We have transforms ready, now we can create our dataset instance
# That will automatically apply the transforms to each image when accessed - as we already defined in our custom dataset class
dataset = AnimeDataset(root_dir=path, transform=transform)

Found 63565 images


In [28]:
# Now I have to create a DataLoader to load the data in batches
# Lets import DataLoader class from torch.utils.data first
from torch.utils.data import DataLoader


In [29]:
# Let's define the dataloader setting batch size, shuffling and num_workers
BATCH_SIZE = 128
SHUFFLE = True
NUM_WORKERS = 4

In [43]:
# Now we are ready to create the train dataloader
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=SHUFFLE, num_workers=NUM_WORKERS)

In [38]:
# We might want to visualize some images from the dataloader to ensure everything is working fine
# Lets install matplotlib if not already installed
! pip install matplotlib
clear_output()


In [ ]:
# Now lets visualize some images from the dataloader
import matplotlib.pyplot as plt
import numpy as np

figure = plt.figure(figsize=(14, 14))
cols, rows = 7, 1
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(dataset), size=(1,)).item()
    img = dataset[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(f"Sample {i}")
    plt.axis("off")
    # Convert from [-1,1] to [0,1] for display and transpose for matplotlib
    img_display = (img + 1) / 2
    img_display = img_display.permute(1, 2, 0)  # CHW to HWC
    plt.imshow(img_display)
plt.show()

## STEP 4: Now we need to build the Generator and Discriminator models.
- Let's start with the Generator model.
- The Generator takes random noise as input and generates fake images.
- We'll use pytorch's `nn.Module` to define our model. More details about `nn.Module` can be found [here](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).

- Let's define the Generator architecture:
  - Input: Random noise vector (latent vector)
  - Layers: A series of transposed convolutional layers (also known as deconvolutional layers) to upsample the noise vector to the desired image size.
  - Output: Generated image

In [46]:
# Lets import nn module from torch to build our models
import torch.nn as nn

In [ ]:
# Generator Class
# - the constructor will define the layers of the model
# - the constructor will take latent_dim as input parameter
# - the forward method will define the forward pass of the model
# - We need to pass the noise vector to the forward method to generate images
class Generator(nn.Module):
    """Generator Model for Vanilla GAN"""
    def __init__(self, z=10, img_channels=3, feature_map_size=64):
        super(Generator, self).__init__()
        self.z = z
        self.img_channels = img_channels
        self.feature_map_size = feature_map_size
        
        # I'll need to define the model architecture here
        # pytorch has a module called `nn.Sequential` which allows us to stack layers sequentially
        self.model = nn.Sequential(
            # Input is the latent vector Z
            # We will use ConvTranspose2d layers to upsample the noise vector
            # Kernel size of 4, stride of 2, padding of 1 -> doubles the spatial dimensions
            # First layer will take the noise vector and produce a feature map of size (feature_map_size*8) x 4 x 4
            nn.ConvTranspose2d(z, feature_map_size * 8, kernel_size=4, stride=1, padding=0, bias=False),
            
            # After each ConvTranspose2d layer, we will use BatchNorm and ReLU activation for non-linearity
            nn.BatchNorm2d(feature_map_size * 8),
            nn.ReLU(True),
            # State size: (feature_map_size*8) x 4 x 4
            
            # Layer 2
            # This will produce a feature map of size (feature_map_size*4) x 8 x 8
            nn.ConvTranspose2d(feature_map_size * 8, feature_map_size * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(feature_map_size * 4),
            nn.ReLU(True),
            # State size: (feature_map_size*4) x 8 x 8
            
            # Layer 3
            # This will produce a feature map of size (feature_map_size*2) x
            nn.ConvTranspose2d(feature_map_size * 4, feature_map_size * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(feature_map_size * 2),
            nn.ReLU(True),
            # State size: (feature_map_size*2) x 16 x 16
            
            # Layer 4
            # This will produce a feature map of size (feature_map_size) x 32 x 32
            nn.ConvTranspose2d(feature_map_size * 2, feature_map_size, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(feature_map_size),
            nn.ReLU(True),
            # State size: (feature_map_size) x 32 x 32
            
            # Final Layer
            # This will produce the final image of size (img_channels) x 64 x 64
            nn.ConvTranspose2d(feature_map_size, img_channels, kernel_size=4, stride=2, padding=1, bias=False),
            
            # Using Tanh activation to ensure the output is in the range [-1, 1] to match the normalized image range
            nn.Tanh()
            # Output size: (img_channels) x 64 x 64
        )
    
    def forward(self, z):
        """Forward pass of the generator.
        Args:
            z (Tensor): Input noise tensor of shape (batch_size, latent_dim, 1, 1)
        Returns:
            Tensor: Generated images of shape (batch_size, img_channels, IMG_SIZE, IMG_SIZE)
        """
        return self.model(z)
    

### Question: What should be the architecture of the Generator model if we want to generate 512x512 images from a 100-dimensional noise vector?

- To generate 512x512 images from a 100-dimensional noise vector, we can use a series of transposed convolutional layers to progressively upsample the noise vector. Here's a possible architecture for the Generator model:

- Input: 100-dimensional noise vector (z)
- Layer 1: Transposed Convolutional Layer
  - Input Channels: 100
  - Output Channels: 512
  - Kernel Size: 4
  - Stride: 1
  - Padding: 0
  - Activation: ReLU
  - Output Size: (512, 4, 4)

- Layer 2: Transposed Convolutional Layer
  - Input Channels: 512
  - Output Channels: 256
  - Kernel Size: 4
  - Stride: 2
  - Padding: 1
  - Activation: ReLU
  - Output Size: (256, 8, 8)

- Layer 3: Transposed Convolutional Layer
  - Input Channels: 256
  - Output Channels: 128
  - Kernel Size: 4
  - Stride: 2
  - Padding: 1
  - Activation: ReLU
  - Output Size: (128, 16, 16)

- Layer 4: Transposed Convolutional Layer
  - Input Channels: 128
  - Output Channels: 64
  - Kernel Size: 4
  - Stride: 2
  - Padding: 1
  - Activation: ReLU
  - Output Size: (64, 32, 32)

- Layer 5: Transposed Convolutional Layer
  - Input Channels: 64
  - Output Channels: 32
  - Kernel Size: 4
  - Stride: 2
  - Padding: 1
  - Activation: ReLU
  - Output Size: (32, 64, 64)

- Layer 6: Transposed Convolutional Layer
  - Input Channels: 32
  - Output Channels: 16
  - Kernel Size: 4
  - Stride: 2
  - Padding: 1
  - Activation: ReLU
  - Output Size: (16, 128, 128)

- Layer 7: Transposed Convolutional Layer
  - Input Channels: 16
  - Output Channels: 8
  - Kernel Size: 4
  - Stride: 2
  - Padding: 1
  - Activation: ReLU
  - Output Size: (8, 256, 256)

- Layer 8: Transposed Convolutional Layer
  - Input Channels: 8
  - Output Channels: 3 (for RGB images)
  - Kernel Size: 4
  - Stride: 2
  - Padding: 1
  - Activation: Tanh (to ensure output values are in the range [-1, 1])
  - Output Size: (3, 512, 512)
  
### Don't forget:
- Forward Method:
  - The forward method will define how the input noise vector is passed through the layers to produce the output image.
  - We need to pass the noise vector to the forward method to generate images

In [48]:
# Now lets move back to the models we were building.
# After the Generator, we need to build the Discriminator model.
# Discriminator Class will be similar to the Generator but in reverse.

# Discriminator Class
# - the constructor will define the layers of the model
# - the constructor will take img_channels as input parameter
# - the forward method will define the forward pass of the model
# - We need to pass the image tensor to the forward method to classify real/fake
class Discriminator(nn.Module):
    """Discriminator Model for Vanilla GAN"""
    def __init__(self, img_channels=3, feature_map_size=64):
        super(Discriminator, self).__init__()
        self.img_channels = img_channels
        self.feature_map_size = feature_map_size
        
        self.model = nn.Sequential(
            # Input is the image tensor (img_channels) x 64 x 64
            # We will use Conv2d layers to downsample the image
            # Kernel size of 4, stride of 2, padding of 1 -> halves the spatial dimensions
            # First layer will take the image and produce a feature map of size (feature_map_size) x 32 x 32
            nn.Conv2d(img_channels, feature_map_size, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # State size: (feature_map_size) x 32 x 32
            
            # Layer 2
            # This will produce a feature map of size (feature_map_size*2) x 16 x 16
            nn.Conv2d(feature_map_size, feature_map_size * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(feature_map_size * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # State size: (feature_map_size*2) x 16 x 16
            
            # Layer 3
            # This will produce a feature map of size (feature_map_size*4) x 8 x 8
            nn.Conv2d(feature_map_size * 2, feature_map_size * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(feature_map_size * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # State size: (feature_map_size*4) x 8 x 8
            
            # Layer 4
            # This will produce a feature map of size (feature_map_size*8) x 4 x 4
            nn.Conv2d(feature_map_size * 4, feature_map_size * 8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(feature_map_size * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # State size: (feature_map_size*8) x 4 x 4
            nn.Conv2d(feature_map_size * 8, 1, kernel_size=4, stride=1, padding=0, bias=False),
            nn.Sigmoid()  # Output a probability between 0 and 1
            # Output size: 1 x 1 x 1 (real/fake probability)
        )
        
    def forward(self, img):
        """Forward pass of the discriminator.
        Args:
            img (Tensor): Input image tensor of shape (batch_size, img_channels, IMG_SIZE, IMG_SIZE)
        Returns:
            Tensor: Probability tensor of shape (batch_size, 1, 1, 1) indicating real/fake
        """
        return self.model(img).view(-1, 1).squeeze(1)  # Flatten output to (batch_size)

# STEP 5: We now need to create objects of the Generator and Discriminator classes and move them to the appropriate device (CPU or GPU).

In [49]:
# Objects of Generator and Discriminator classes
# - We now need to create objects of the Generator and Discriminator classes and move them to the appropriate device (CPU or GPU).
# - We will also print the model summaries to verify everything is correct
# - We will use a latent dimension of 100 for the Generator
latent_dim = 100
img_channels = 3
feature_map_size = 64

G = Generator(z=latent_dim, img_channels=img_channels, feature_map_size=feature_map_size)
G.to(device) # Move Generator object to device

D = Discriminator(img_channels=img_channels, feature_map_size=feature_map_size)
D.to(device) # Move Discriminator object to device

Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [51]:
# To print the model summaries, we can use the `summary` function from `torchsummary` package
! pip install torchsummary
clear_output()

In [55]:
# Lets import summary function from torchsummary
import numpy
from torchsummary import summary

In [ ]:
# Print Generator summary
summary(G, (latent_dim, 1, 1))

In [ ]:
# Discriminator summary
summary(D, (3, IMG_SIZE, IMG_SIZE))

## STEP 6: Training the GAN
- Now that we have defined our Generator and Discriminator models, we need to train them.
- The training process involves alternating between training the Discriminator and the Generator.

### Step 6.1: Train the Discriminator
The discriminator's goal is to get better at distinguishing real images from fakes. Its total loss is the sum of its performance on real and fake images.
$$ \nabla_{\theta_d} \frac{1}{m} \sum_{i=1}^{m} [\log D(x^{(i)}) + \log(1 - D(G(z^{(i)})))] $$
- We feed it a batch of **real images** and train it to classify them as `real` (label=1).
- We then feed it a batch of **fake images** (generated by the generator) and train it to classify them as `fake` (label=0).

### Step 6.2: Train the Generator
The generator's goal is to fool the discriminator. It wants the discriminator to classify its fake images as real.
$$ \nabla_{\theta_g} \frac{1}{m} \sum_{i=1}^{m} \log(D(G(z^{(i)}))) $$
- We generate a new batch of **fake images**.
- We pass them through the discriminator, but this time, we use `real` labels (label=1).
- We calculate the loss and backpropagate it to update **only the generator's weights**.

### Training Loop/Algorithm
- We would need to define the number of epochs - how many times we want to go through the entire dataset again and again.
- For each epoch, we would iterate through the dataset in batches - for each batch, we would perform the steps to train the discriminator and generator.
- But before that, we need to define the loss function and optimizers for both models. This is typically done using Binary Cross Entropy Loss and Adam optimizer.

#### Bianry Cross Entropy Loss
- We can use `nn.BCELoss()` from PyTorch to define the binary cross-entropy loss function.

#### Adam Optimizer
- We can use `torch.optim.Adam` to define the Adam optimizer for both the generator and discriminator.

In [ ]:
# Define Loss function and Optimizers
# - We will use Binary Cross Entropy Loss for both models
# - We will use Adam optimizer for both models
# - We will set the learning rate to 0.0001 and betas to (0.5, 0.999)
# - Betas are hyperparameters for the Adam optimizer that control the exponential decay rates of moving averages of past gradients and squared gradients.
# - Beta1 (0.5) is used for the first moment estimate (mean of gradients) and helps to smooth out the updates.
# - Beta2 (0.999) is used for the second moment estimate (uncentered variance of gradients) and helps to stabilize the training by reducing the variance of the updates.
# - These values are commonly used in GAN training to help with convergence and stability.
# - We will define separate optimizers for the Generator and Discriminator
import torch.optim as optim
criterion = nn.BCELoss()
lr = 0.0001
betas = (0.5, 0.999)
optimizer_G = optim.Adam(G.parameters(), lr=lr, betas=betas)
optimizer_D = optim.Adam(D.parameters(), lr=lr, betas=betas)

In [ ]:
# Training Loop/Algorithm
# - We would need to define the number of epochs - how many times we want to go through the entire dataset again and again.
# - For each epoch, we would iterate through the dataset in batches - for each batch, we would perform the steps to train the discriminator and generator.
epochs = 100

from tqdm import tqdm

# Lists to keep track of progress
G_losses = []
D_losses = []

# Epoch loop
for i in range(epochs):
    # Let's show the progress of training
    pbar = tqdm(dataloader, desc=f"Epoch {i+1}/{epochs}")
    for batch_idx, real_images in enumerate(pbar):
        # STEP 1: Get the batch size
        batch_size = real_images.size(0)
        # STEP 2: Move real images to device
        real_images = real_images.to(device)
        
        # So we have real images in the range [-1, 1] as we normalized them earlier
        # We need to create labels for real and fake images
        # Real images are labeled as 1 and fake images are labeled as 0
        # We will use these labels to compute the loss for the discriminator

        """
        Train Discriminator
        """
        optimizer_D.zero_grad() # Zero the gradients for the discriminator
        
        # Discriminator will be trained on both real and fake images
        # What we do as steps:
        # - Label real images as 1
        # - Forward pass real images through D - get D's output
        # - Calculate loss on real images
        # Real images
        labels_real = torch.ones(batch_size, device=device) # Loaded dataset images are real images - hence labeled as 1
        output_real = D(real_images) # Forward pass real batch through D
        loss_real = criterion(output_real, labels_real) # Calculate loss on all-real batch
        
        # Now we need to generate fake images using the generator
        # - We will sample random noise vectors from a normal distribution
        # - We will pass these noise vectors to the generator to generate fake images
        # - We will label these fake images as 0
        # - We will forward pass these fake images through the discriminator to get D's output
        # - We will calculate the loss on fake images
        # Fake images
        noise = torch.randn(batch_size, latent_dim, 1, 1, device=device)
        fake_images = G(noise)
        labels_fake = torch.zeros(batch_size, device=device)
        output_fake = D(fake_images.detach())  # Detach to avoid training G on these labels
        loss_fake = criterion(output_fake, labels_fake)
        
        # So in the GAN literature, the discriminator's loss is typically computed as the sum of the losses on real and fake images.
        # That way, the discriminator is penalized for misclassifying both real and fake images.
        # Total Discriminator loss
        loss_D = loss_real + loss_fake
        
        # Once the loss is computed, we perform backpropagation and update the discriminator's weights
        loss_D.backward()
        # We then update the weights of the discriminator using the optimizer
        optimizer_D.step()


        """
        Train Generator
        """
        # We want to train the generator to fool the discriminator
        # At the same time we want to keep the last trained discriminator's weights fixed
        optimizer_G.zero_grad() # Zero the gradients for the generator
        
        # We want the generator to fool the discriminator
        # Generator will fool the discriminator if the discriminator classifies the fake images as real
        # So we will label the fake images as real (1) and compute the loss
        # We will then backpropagate this loss through the generator to update its weights
        labels_gen = torch.ones(batch_size, device=device)  # We want the fake images to be classified as real
        output_gen = D(fake_images) # Forward pass fake images through D
        loss_G = criterion(output_gen, labels_gen) # Calculate loss on fake images
        
        # Backpropagate the loss through the generator
        loss_G.backward()
        # Update the weights of the generator using the optimizer
        optimizer_G.step()
        
        # Save the losses for plotting later
        G_losses.append(loss_G.item())
        D_losses.append(loss_D.item())
        
        # Update the progress bar
        pbar.set_postfix({"Loss D": loss_D.item(), "Loss G": loss_G.item()})
    pbar.close()

    print(f"Epoch [{i+1}/{epochs}]  Loss D: {loss_D.item():.4f}, Loss G: {loss_G.item():.4f}")

## STEP 7: Monitoring Progress
- During training, it's important to monitor the progress of both the generator and discriminator.
- We can do this by printing the losses of both models after each epoch.
- Additionally, we can generate and save some sample images from the generator at regular intervals (e.g., every 10 epochs) to visually inspect the quality of the generated images over time.
- We can use libraries like Matplotlib to display and save these images.

### Training Loss Visualization

Plotting the losses helps us understand the dynamics of the adversarial training. 
- **Discriminator Loss (D)**: Ideally, this should hover around 0.5. If it goes to 0, the generator isn't learning.
- **Generator Loss (G)**: We want to see this loss decrease, indicating the generator is getting better at fooling the discriminator.

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# We can save the trained models for future use
torch.save(G.state_dict(), "generator.pth")
torch.save(D.state_dict(), "discriminator.pth")

In [ ]:
# Now we can generate some images using the trained generator
# We will sample random noise vectors and pass them to the generator to generate images
# We will then visualize these generated images
import matplotlib.pyplot as plt
import numpy as np
# Set the generator to evaluation mode
G.eval()

In [ ]:
# Now sample some random noise vectors
num_samples = 10
noise = torch.randn(num_samples, latent_dim, 1, 1, device=device)
with torch.no_grad():
    generated_images = G(noise).cpu()  # Generate images and move to CPU
# Plot the generated images
plt.figure(figsize=(14, 14))
cols, rows = 5, 2
for i in range(1, cols * rows + 1):
    figure.add_subplot(rows, cols, i)
    plt.title(f"Generated {i}")
    plt.axis("off")
    # Convert from [-1,1] to [0,1] for display and transpose for matplotlib
    img_display = (generated_images[i-1] + 1) / 2
    img_display = img_display.permute(1, 2, 0)  # CHW to HWC
    plt.imshow(img_display)
plt.show()